# LS Sitting Members Data Cleaning & Data Updating Script

In [1]:
print ('Initiating Script for Cleaning Lok Sabha Members Data\nImporting Libraries')
import json
import pymongo
import pandas as pd
from collections import defaultdict
from datetime import datetime
from itertools import permutations

Initiating Script for Cleaning Lok Sabha Members Data
Importing Libraries


#Connecting to the Mongo Database and respective collections

In [2]:
print ('Connecting to the mongo_database with details provided in file "config.cfg"')
config_file = open("../config.cfg")
config = json.load(config_file)
client = pymongo.MongoClient(config["mongodb_uri"])
db = client[config["database"]]
members_collection = db['current_ls_members']
constituencies = db['constituencies']
political_parties = db['political_parties']
archive_members = db["archive_ls_members"]
print ('the database connected now is',config["database"])
print ('the data collection loaded is:\n',members_collection)

Connecting to the mongo_database with details provided in file "config.cfg"
the database connected now is factly_parliament_search
the data collection loaded is:
 Collection(Database(MongoClient(host=['35.200.213.251:27017'], document_class=dict, tz_aware=False, connect=True), 'factly_parliament_search'), 'current_ls_members')


In [3]:
#mongodb collections to pandas dataframes for members, constituencies and parties
df_m = pd.DataFrame(list(members_collection.find({})))
df_c = pd.DataFrame(list(constituencies.find({})))
df_p = pd.DataFrame(list(political_parties.find({})))
df_a = pd.DataFrame(list(archive_members.find({})))

# Script to change Sessions into an array of dictionary

In [20]:
for row in current_members.find({}):
    if archive_members.count_documents({"_id":row["_id"]}) > 0:
        continue
        count = count+1
        old_row = archive_members.find_one({"_id":row["_id"]})
        current_sessions = []
        for session in old_row["session"]:
            current_sessions.append({"session" : session,
                                    "party_id" : "NA",
                                    "constituency_id" : "NA"
                                   })
        current_sessions.append({"session" : "17",
                                "party_id" : row["party"],
                                "constituency_id" : row["constituency"]
                               })
        row["session"] = current_sessions
        members_collection.update_one({"_id":row["_id"]},{"$set":{"session":current_sessions}})
    else:
        current_sessions = []
        current_sessions.append({"session" : "17",
                                "party_id" : row["party"],
                                "constituency_id" : row["constituency"]
                               })
        members_collection.update_one({"_id":row["_id"]},{"$set":{"session":current_sessions}})

NameError: name 'current_members' is not defined

# Defining Functions for data cleaning

In [6]:
#Name Cleaning Function "name_clean()"
def name_clean(member_name):
    member_name = member_name.strip()
    if 'Sh.' in member_name:
        member_list = member_name.split('Sh.')
        name = 'Sh.'+" "+member_list[-1]+" "+' '.join(member_list[:-1])
    elif 'Smt.' in member_name:
        if "," in member_name: 
            name_list = member_name.split(',')
            if len(name_list) >= 2:
                name = name_list[1].strip()+" "+name_list[0].strip()
                name = name.replace("  "," ").replace('\xa0','')
        else:
            member_list = member_name.split('Smt.')
            name = 'Smt.'+" "+member_list[-1]+" "+' '.join(member_list[:-1]).strip(',')
    elif ',' in member_name:
        name_list = member_name.split(',')
        if len(name_list) >= 2:
            name = name_list[1].strip()+" "+name_list[0].strip()
            name = name.replace("  "," ").replace('\xa0','')
    else:
        name = member_name
    return name

#Gender creating Function "get_gender()"
def get_gender(name):
    title = name
    female_titles = [
        'Begam',
        'Begum',
        'Devi',
        'Dr.(Kum.)',
        'Dr.(Smt.)',
        'Dr.(Smt.)Chellamalla',
        'Her',
        'Km.',
        'Kum.',
        'Kumari',
        'Maharani',
        'Ms.',
        'Prof.(Smt.)',
        'Rajkumari',
        'Rajmata',
        'Rajmata(Jodhpur)',
        'Rani',
        'Sadhvi',
        'Sardarni',
        'Smt',
        'Smt.',
        'Sushree',
        'Sukanta Majumdar',
        'Rita Bahuguna Joshi',
        'Heena Vijaykumar Gavit',
        'Bharatiben Dhirubhai Shyal',
        'Beesetti Venkata Satyavathi',
        'Dr. Bharti Pravin Pawar',
        'Dr. T. Sumathy'
    ]
    male_titles = [
        '(Pandit)',
        'Acharya',
        'Adv.',
        'Advocate',
        'Baba',
        'Babu',
        'Capt.',
        'Capt.(Retd.)',
        'Captain',
        'Ch.',
        'Chaudhari',
        'Chaudhary',
        'Chaudhri',
        'Chaudhry',
        'Choudari',
        'Choudhary',
        'Choudhury',
        'Chowdhary',
        'Chowdhry',
        'Chowdhury',
        'Col.',
        'Dr.Maharaja',
        'Father',
        'General',
        'H.H.',
        'H.H.Maharaja',
        'Haji',
        'Kazi',
        'Kh.',
        'Kunwar',
        'Lala',
        'Lt.',
        'Lt.Col.',
        'Mahant',
        'Maharajkumar',
        'Maj',
        'Maj.',
        'Maj.Gen.',
        'Major',
        'Major-General',
        'Maulana',
        'Mohammad',
        'Mohd.',
        'Motilal',
        'Mulla',
        'Pandit',
        'Purohit',
        'Qazi',
        'Raja',
        'Rt.',
        'Sardar',
        'Seth',
        'Shaikh',
        'Shri',
        'Shriman',
        'Swami',
        'Syed',
        'Thakur',
        'Thiru',
        'Vaidya',
        'Visharad',
        'Farooq Abdullah',
        'Vishnu Prasad',
        'Radhakrishna Vikhepatil',
        'Shashi Tharoor',
        'Maharaj',
        'Dr. Alok Kumar Suman',
        'Dr. (Prof.) Kirit Premjibhai Solanki',
        'Satya Pal Singh',
        'Dr. Jitendra Singh',
        'Dr. Amar Singh',
        'Dr. Sanjeev Kumar Singari',
        'Dr. Shrikant Eknath Shinde',
        'Dr. Mahesh Sharma',
        'Dr. Arvind Kumar Sharma',
        'Dr. Jadon Chandra Sen',
        'Dr. Subhas Sarkar',
        'Prof. Sougata Ray',
        'Dr. Jayanta Kumar Roy',
        'Dr. Rajdeep Roy',
        'Dr. Gaddam Ranjith Reddy',
        'Rao Inderjit Singh',
        'Dr. Manoj Rajoria',
        'Dr. Ranjan Singh Rajkumar',
        'Dr. Lorho S. Pfoze',
        'Dr. K. C. Patel',
        'Dr. Mahendra Nath Pandey',
        'Dr. T. R. Paarivendhar',
        'Dr. Ramesh Pokhriyal Nishank',
        'Dr. Mahendrabhai Kalubhai Munjpara',
        'Dr. Pritam Gopinath Munde',
        'Dr. Sanghamitra Maurya',
        'Dr. Jaisiddeshwar Shivacharya Mahaswamiji',
        'Dr. Virendra Kumar',
        'Dr. Amol Ramsing Kolhe',
        'Prof. (Dr.) Ram Shankar Katheria',
        'Dr. V. Kalanidhi',
        'Dr. Mohammad Jawed',
        'Dr. K. Jayakumar',
        'Dr. Sanjay Jaiswal',
        'Dr. Umesh G. Jadhav',
        'Dr. Harsh Vardhan',
        'Dr. S.T. Hasan',
        'Vijay Kumar Singh (Retd.) General (Dr.)',
        'Dr. Nishikant Dubey',
        'Dr. A. Chellakumar',
        'Dr. Dhal Singh Bisen',
        'Dr. Subhash Ramrao Bhamre',
        'Dr. Sanjeev Kumar Balyan',
        'Prof. S.P. Singh Baghel'
    ]
    if any(x in title for x in female_titles):
        return "Female"
    if any(x in title for x in male_titles):
        return "Male"
    else:
        return ""

#Date Cleaning Function "date_clean()"

def date_clean(dat):
    date_formats = ["%d.%m.%Y", "%d %b %Y", "%B %d, %Y","%Y-%m-%d","%d %B,% %Y","%d %B %Y","%Y-%b-%d","%Y-%m-%d"]
    for date_format in date_formats:
        try:
            return datetime.strptime(dat,date_format).strftime("%Y-%m-%d")
        except:
            pass
    return dat

#education cleaning Function "edu_clean()"

def edu_clean(edu):
    #list of values for education level for the function to check for
    phd =['Ph.D.','Ph. D', 'Doctorate']
    ug = ['Engg','Bachelor of Engineering','B. A.','B.A','B.Com','B.Sc','L.L.B','B.E','B.B.M',
          'B.B.A','M.B.B.S','B.M.S.','C.A', 'B. Com','B. Sc','Undergraduate','Law','B. Tech.',
          'Universit','B.Tech.','B. Tech','Graduat']
    pg = ['LL.M','M.A','M.Sc','M.Com','M.B.A','M.D','M.D.M','M.E','M.L','L.L.M','Graduate','Post Graduate','M. Com','Master','Masters']
    inter =['Inter','Intermediate','Higher Secondary','PUC']
    school = ['High School', 'S.S.C', 'School','school']
    undermatric = ['Under Matriculate','Under-Matric','Under Matric']
    matric=['Matric','Matriculation']
    
    if any(x in edu for x in phd):
        return "Doctorate"
    elif any(x in edu for x in pg):
        return "Post Graduate"
    elif any(x in edu for x in ug):
        return "Graduate"
    elif any(x in edu for x in inter):
        return "Intermediate"
    elif any(x in edu for x in school):
        return "High School"
    elif any(x in edu for x in matric):
        return "Matriculate"
    elif any(x in edu for x in undermatric):
        return "Under Matriculate"
    elif "Diploma" in edu:
        return "Diploma"
    else:
        return ""

In [7]:
#implementing name_cleaning
for i,row in df_m.iterrows():
    df_m.loc[i,"name"] = name_clean(row["name"])

print ('gender cleaning')
#adding gender column
df_m["gender"] = ""
#implementing gender cleaning
for i,row in df_m.iterrows():
    df_m.loc[i,"gender"] = get_gender(row["name"])
print("Number of records where Gender is not available is :", len(df_m.loc[df_m['gender']==""]))

#Implementing date of birth Cleaning
for i,row in df_m.iterrows():
    df_m.loc[i,"dob"] = date_clean(row["dob"])

print ('Date of Birth Cleaning completed')
print("Number of records where DOB is not available is :", len(df_m.loc[df_m['dob']==""]))

#implementing education level cleaning
df_m["edu_level"] = ""
for i,row in df_m.iterrows():
    df_m.loc[i,"edu_level"] = edu_clean(str(row["education"]))

print("Number of records where Education Level is not available is:", len(df_m.loc[df_m['edu_level']==""]))


gender cleaning
Number of records where Gender is not available is : 0
Date of Birth Cleaning completed
Number of records where DOB is not available is : 0
Number of records where Education Level is not available is: 169


# Profession Cleaning

In [8]:
# Dictionary of professions
prof_dict = {
    "Agriculturist" : "Agriculturist",
    "agriculturist" : "Agriculturist",
    "Advocate" : "Lawyer",
    "Adviser" : "Advisor",
    "Animal husbandry" : "Animal husbandry",
    "author" : "Author",
    "Ayurvedic" : "Ayurved",
    "Bar-at-Law" : "Lawyer",
    "Barrister-at-Law" : "Lawyer",
    "Banker" : "Banker",
    "Bharatiya Janata party" : "Political worker",
    "Builder" : "Builder",
    "Building and Road Contractor" : "Contractor",
    "Businessman" : "Business",
    "Businessperson" : "Business",
    "Chariman" : "Chairman",
    "Chartered Accountant" : "Chartered Accountant",
    "Cine Exhibitor" : "Cine Exhibitor",
    "Civil Servant" : "Civil Servant",
    "Civil Service" : "Civil Servant",
    "Civil service" : "Civil Servant",
    "Comedian" : "Comedian",
    "Commerce" : "Commerce",
    "Communist" : "Political worker",
    "Congress" : "Political worker",
    "Consultant" : "Consultant",
    "Consulting" : "Consultant",
    "C.P.I" : "Political worker",
    "Cultivator" : "Cultivator",
    "Defence Services" : "Defence",
    "Diplomat" : "Diplomat",
    "Doctor" : "Doctor",
    "Economist" : "Economist",
    "educationist" : "Educationist",
    "Educationist" : "Educationist",
    "editor" : "Editor",
    "Editor" : "Editor",
    "Engineer" : "Engineer",
    "Entrepreneur" : "Entrepreneur",
    "Ex-Governor" : "Ex-Governor",
    "Ex-Commissioned Officer" : "Ex-Commissioned Officer",
    "Exporter" : "Exporter",
    "Farming" : "Farmer",
    "Farmer" : "Farmer",
    "Film Artist" : "Film Artist",
    "Film Producer" : "Film Producer",
    "Former Judge" : "Former Judge",
    "Founder" : "Founder",
    "Founded" : "Founder",
    "Government Servant" : "Civil Servant",
    "Harijan" : "Political worker",
    "Homeopath" : "Homeopath",
    "Horticulturist" : "Horticulturist",
    "IAS Officer" : "IAS Officer",
    "Administrative Service Officer" : "IAS Officer",
    "I.T. Professional" : "I.T. Professional",
    "Imprison" : "Imprisoned",
    "imprisonment" : "Imprisoned",
    "Industrialist" : "Industrialist",
    "IPS" : "IPS Officer",
    "Journalist" : "Journalist",
    "L.I.C." : "L.I.C Agent",
    "Lecturer" : "Lecturer",
    "Landlord" : "Landlord",
    "lawyer" : "lawyer",
    "Lawyer" : "Lawyer",
    "legal practice" : "Lawyer",
    "Legal practitioner" : "Lawyer",
    "Legal Practitioner" : "Lawyer",
    "Legislative Assembly" : "MLA",
    "Literateur" : "Literateur",
    "Lok Sabha" : "Lok Sabha member",
    "LL.B" : "Lawyer",
    "Managing Director" : "Managing Director",
    "Merchant" : "Merchant",
    "merchant" : "Merchant",
    "Medical Practitioner" : "Medical Practitioner",
    "Midwife" : "Midwife",
    "Military Service" : "Military",
    "Millowner" : "Mill owner",
    "Mine owner" : "Mine owner",
    "Minister for Education" : "Education Minister",
    "Minister" : "Minister",
    "Ministry of Irrigation and Power" : "Ministry of Irrigation and Power",
    "Musician" : "Musician",
    "Orator" : "Orator",
    "Performing Artiste" : "Artist",
    "Pilot" : "Pilot",
    "Pleader" : "Lawyer",
    "Police Service" : "Police",
    "Politics" : "Politician",
    "Political worker" : "Political worker",
    "Political Worker" : "Political worker",
    "Political" : "Political worker",
    "Politician" : "Politician",
    "Poet" : "Poet",
    "Poultry farming" : "Poultry farming",
    "Philanthropist" : "Philanthropist",
    "Physician" : "Physician",
    "Principal" : "Principal",
    "Professor" : "Professor",
    "professor" : "Professor",
    "Producer and Director" : "Producer and Director",
    "publisher" : "Publisher",
    "Public Worker" : "Public Worker",
    "Public worker" : "Public Worker",
    "Real Estate Development" : "Real Estate Development",
    "Religious Missonary" : "Religious Missonary",
    "Religious missonary" : "Religious Missonary",
    "Royal Indian Navy" : "Royal Indian Navy",
    "Secretary" : "Secretary",
    "Singer" : "Singer",
    "Social Worker" : "Social worker",
    "Social worker" : "Social worker",
    "Solicitor" : "Lawyer",
    "Sportsman" : "Sportsperson",
    "Sportsperson" : "Sportsperson",
    "Surgeon" : "Surgeon",
    "teacher" : "Teacher",
    "Teacher" : "Teacher",
    "Telangana Praja Samithi" : "Political worker",
    "Technologist" : "Technologist",
    "Trader" : "Trader",
    "Trade Unionist" : "Trade Union",
    "Trade Union" : "Trade Union",
    "Treasurer" : "Treasurer",
    "Vakil" : "Lawyer",
    "Veterinarian" : "Veterinarian",
    "Weaver" : "Weaver",
    "Widow" : "Widow",
    "widower" : "Widow",
    "Writer" : "Writer",
    "writer" : "Writer",
    "Worked among the backward Muslim communities" : "Community helper",
    "Zamindar" : "Landlord"
}

In [9]:
# Copy profession values to a new column
df_m["new_profession"] = df_m['profession']
# Splitting the string into a series
df_m.new_profession = df_m.new_profession.astype(str)

# Removing return carriage and newline
df_m.new_profession = df_m.new_profession.str.replace(r'\r\n', '')
# Removing _x000D_ in the string series
df_m.new_profession = df_m.new_profession.str.replace(r'_x000D_', '')
# Replacing multiple spaces with a single space
df_m.new_profession = df_m.new_profession.str.replace(r'\s+', ' ')

# Making new_profession in the format ['Profession1', 'Profession2'] format
sent_set_list = []
for sent in df_m.new_profession:
    if (sent == 'nan'):
        sent_set_list.append([])
        continue
    string_prof = set()
    for (prof, prof_name) in prof_dict.items():
        if prof in sent:
            string_prof.add(prof_name)
    sent_set_list.append(list(string_prof))

df_m.new_profession = sent_set_list
count = 0
for cnan in sent_set_list:        
    if 'nan' in cnan:
        count +=1
print ("Number of records where Profession is not available is:", count)

Number of records where Profession is not available is: 0


# Email Cleaning

In [10]:
#Defining a new column for cleaned email id's as a array of lists
for i,row in df_m.iterrows():
    df_m.loc[i,"new_email"] = ['']
    
#Function for replacing the [AT] and [DOT] in the emails
dic = {
    "[AT]" : "@",
    "[DOT]" : ".",
    "NIL" : ""
    }
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

#Iterating over each row of emails and appending them to the new_email
for i,row in df_m.iterrows():
    row = df_m.loc[i, "email"]
    new_row = []
    for j in row:
        new_row.append(replace_all(j,dic))
        j.replace(",","")
    df_m.at[i,"new_email"] = new_row

print("Number of records where Email is not available is:", len(df_m.loc[df_m['new_email']==""]))

Number of records where Email is not available is: 0


# Term Details : session, Party_id, Constituency_id, house

In [11]:
for i, row in df_m.iterrows():
    print(i)
    row = df_m.loc[i,"terms"]
    for j in row:
        for key, value in j.items():
            for k, l in df_p.iterrows():
                l = df_p.loc[k,"party_raw"]
                if j['party_id'] == l:
                    j['party_id'] = df_p.loc[k,"_id"]
            for m, n in df_c.iterrows():
                n = df_c.loc[m,"constituency"]
                if j['constituency_id'] == n:
                    j['constituency_id'] = df_c.loc[m,"_id"]

KeyboardInterrupt: 

# creating dataframe with required columns


In [12]:
members = df_m[['_id','name','gender','dob','birth_place','marital_status',
                'sons','daughters','new_email','phone','new_profession','edu_level','terms']]

In [13]:
members = members.rename(columns={"_id":"MemberId","new_email":"EmailId",
                                  "new_profession":"Profession","edu_level":"Education",
                                 "gender":"Gender","birth_place":"BirthPlace","marital_status":"MaritalStatus",
                                 "name":"Name","dob":"Dob","sons":"Sons","daughters":"Daughters","phone":"Phone",
                                 "terms":"Terms"})

In [14]:
for i,row in members.iterrows():
    members.loc[i,"Expertise"] = ['']

# Adding the members dataframe to MongoDb Collection

,0,1,2,3,4,5,6,7,8,9,...,532,533,534,535,536,537,538,539,540,541
MemberId,5175,4498,4843,38,5076,4921,5101,5008,3654,5007,...,5180,4436,4013,4944,4793,4267,4502,4939,4847,2654
Name,Shri Narayana Swamy Abbaiah,Shri Kalyan Banerjee,Shri Abhishek Banerjee,Shri Sudip Bandyopadhyay,Shri Sanjay Kumar Bandi,Dr. Sanjeev Kumar Balyan,Shri Durga Prasad Rao Balli,Shri Deepak Baij,Shri Subhash Chandra Baheria,Shri Vijay Baghel,...,Smt. Sumalatha Ambareesh,Maulana Badruddin Ajmal,Shri Guharam Ajgalley,Smt. Sajda Ahmed,Shri S.S. Ahluwalia,Shri Rajendra Agrawal,Shri Sisir Kumar Adhikari,Shri Dibyendu Adhikari,Shri Deepak (Dev) Adhikari,Dr. Farooq Abdullah
Gender,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,...,Female,Male,Male,Female,Male,Male,Male,Male,Male,Male
Dob,1957-05-16,1957-01-04,1987-11-07,1949-01-06,1971-07-11,1972-06-23,1956-06-15,1981-07-14,1956-12-17,1959-08-15,...,1963-08-27,1956-02-12,1967-04-30,1962-06-22,1951-07-04,1951-10-02,1941-09-19,1976-12-24,1982-12-25,1937-10-21
BirthPlace,"Anekal, Bangaluru, Karnataka","Asansol, Burdwan (West Bengal)","Kolkata, West Bengal","Berhampore, Distt. Murshidabad (West Bengal)",NaN,"Vill. Kutbi, Distt. Muzaffarnagar (Uttar Pradesh)","Venkata Giri, Nellore, Andhra Pradesh",NaN,"Shahpura, Bhilwara, Rajasthan",NaN,...,NaN,"Gopal Nagar, Nagaon (Assam)",Chhote Khiara Distt. Raigarh (Chhattisgarh),"Kolkata, Kolkata, West Bengal","Jaykay Nagar, Distt. Burdwan (West Bengal)","Pilkhuwa, Distt. Ghaziabad (Uttar Pradesh)","Karkuli (Purba Medinipur), West Bengal","Contai, Purba Medinipur, West Bengal","Mumbai, Maharashtra",Srinagar ( Jammu & Kashmir)
MaritalStatus,Married,Married,Married,Married,NaN,Married,Married,NaN,Married,NaN,...,NaN,Married,MARRIED,Widow,Married,Married,Married,Married,Unmarried,Married
Sons,NaN,1,NaN,NaN,NaN,NaN,2,NaN,1,NaN,...,NaN,6,2,2,3,1,4,NaN,NaN,1
Daughters,3,1,1,NaN,NaN,2,1,NaN,2,NaN,...,NaN,1,3,NaN,2,1,NaN,1,NaN,3
EmailId,"[anarayanaswamyanekal5@gmail.com, answamydhurg...","[banerjee.kalyan@sansad.nic.in, kalyanbanerjee...",[abhishekbanerjee.mp@gmail.com],[sudip@sansad.nic.in sudip_bandyopadhyay2006@y...,[bandisanjayindian@gmail.com],"[sanjeevkumar.balyan@sansad.nic.in, drbalyan@r...",[bdpysr@gmail.com],[deepakbaij77727@gmail.com],"[subhash.baheria@sansad.nic.in, baheriasc@gmai...",[vijay.baghel1503@gmail.com],...,[sumabi@yahoo.com],"[b.ajmal@sansad.nic.in, amarhuzoor@gmail.com]",[ajgalley1969@gmail.com],"[sajda.ahmed@sansad.nic.in, sajdamp@gmail.com]",[ssa@sansad.nic.in],"[rajendra.agrawal51@gmail.com, rajendra.agrawa...","[sisir.adhikari@nic.in, sisiradhikari76@yahoo....",[dibyendu.adhikari27@yahoo.com],"[adhikari.deepak@sansad.nic.in, medevadhikari@...",[iamfarooq70@hotmail.com]
Phone,[09900845555],"[09013180097, 09830035733, 0943267130]","[09830510000, 09434145333]","[09433086118, 09013180066]",[9885289261],[0847705777],[09866369499],"[08817277727, 09406077448]",[09829046344],[08770360478],...,[09845052494],[09013180448],[07771863377],[09836390227],[09868181816],[09412202623],"[09434039494, 0965008025]","[09434005207, 09013869518]",[09013869520],"[09018210000, 09810099847]"


In [44]:
for i, row in members.T.iterrows():
    try:
        row.to_json()
    except:
        print(i, row)

Terms 0      [{'session': '17', 'state': 'Karnataka', 'hous...
1      [{'session': '15', 'house': 'Lok Sabha', 'part...
2      [{'session': '16', 'house': 'Lok Sabha', 'part...
3      [{'session': '12', 'house': 'Lok Sabha', 'part...
4      [{'session': '17', 'state': 'Telangana', 'hous...
5      [{'session': '16', 'house': 'Lok Sabha', 'part...
6      [{'session': '17', 'state': 'Andhra Pradesh', ...
7      [{'session': '17', 'state': 'Chhattisgarh', 'h...
8      [{'session': '11', 'house': 'Lok Sabha', 'part...
9      [{'session': '17', 'state': 'Chhattisgarh', 'h...
10     [{'session': '16', 'house': 'Lok Sabha', 'part...
11     [{'session': '10', 'house': 'Lok Sabha', 'part...
12     [{'session': '11', 'house': 'Lok Sabha', 'part...
13     [{'session': '17', 'state': 'Madhya Pradesh', ...
14     [{'session': '11', 'house': 'Lok Sabha', 'part...
15     [{'session': '11', 'house': 'Lok Sabha', 'part...
16     [{'session': '17', 'state': 'Bihar', 'house': ...
17     [{'session': '13',

In [64]:
members.T[0]

MemberId                                                      5175
Name                                   Shri Narayana Swamy Abbaiah
Gender                                                        Male
Dob                                                     1957-05-16
BirthPlace                            Anekal, Bangaluru, Karnataka
MaritalStatus                                              Married
Sons                                                           NaN
Daughters                                                        3
EmailId          [anarayanaswamyanekal5@gmail.com, answamydhurg...
Phone                                                [09900845555]
Profession                               [Business, Social worker]
Education                                                 Graduate
Terms            [{'session': '17', 'state': 'Karnataka', 'hous...
Expertise                                                         
Name: 0, dtype: object

In [ ]:
["MemberId","Name","Gender","Dob","BirthPlace","MaritalStatus",
              "Sons","Daughters","EmailId","Phone","Profession","Education","Terms","Expertise"]

In [90]:
recor = members.T[0]

In [101]:
recor[['Terms']]

Terms    [{'session': '17', 'state': 'Karnataka', 'hous...
Name: 0, dtype: object

In [95]:
recor['Terms'][0]

{'session': '17',
 'state': 'Karnataka',
 'house': 'Lok Sabha',
 'party_id': ObjectId('5d72435233ffb3f65e04dcee'),
 'constituency_id': 1171}

Run this only after you check the dataframe

In [62]:
records = json.loads(members.T.to_json(force_ascii= False)).values()
db["cleaned_members_data"].insert(records)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 457: invalid start byte